<a href="https://colab.research.google.com/github/doronin99/RecoServiceTemplate/blob/task3/notebooks/userknn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing necessary libraries

In [ ]:
pip install -q implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 56.6 MB/s eta 0:00:00


In [ ]:
pip install -q rectools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 2.4 MB/s eta 0:00:00


In [ ]:
pip install -q dill

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 1.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import requests

from joblib import Parallel, delayed
from tqdm.auto import tqdm
from scipy.stats import mode
from pprint import pprint
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender
import warnings
import dill

from rectools import Columns
from rectools.dataset import Dataset, Interactions
from rectools.metrics import MAP, Serendipity, MeanInvUserFreq, calc_metrics, Precision, Recall
from rectools.model_selection import TimeRangeSplitter

from typing import Dict, List
from collections import Counter

## Data loading

In [ ]:
!wget -q https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip -O data_original.zip
!unzip -o data_original.zip
!rm data_original.zip

Archive:  data_original.zip
   creating: data_original/
  inflating: data_original/interactions.csv  
  inflating: __MACOSX/data_original/._interactions.csv  
  inflating: data_original/users.csv  
  inflating: __MACOSX/data_original/._users.csv  
  inflating: data_original/items.csv  
  inflating: __MACOSX/data_original/._items.csv  


In [ ]:
interactions = pd.read_csv('data_original/interactions.csv')
users = pd.read_csv('data_original/users.csv')
items = pd.read_csv('data_original/items.csv')

## EDA

### interactions

In [ ]:
interactions_df = interactions.rename(columns={'total_dur': Columns.Weight,
                                            'last_watch_dt': Columns.Datetime})

interactions_df['datetime'] = pd.to_datetime(interactions_df['datetime'])

In [ ]:
interactions = Interactions(interactions_df)

In [ ]:
pd.concat([interactions_df.head(), interactions_df.tail()])

user_id  item_id   datetime   weight  watched_pct
0         176549     9506 2021-05-11   4250.0         72.0
1         699317     1659 2021-05-29   8317.0        100.0
2         656683     7107 2021-05-09     10.0          0.0
3         864613     7638 2021-07-05  14483.0        100.0
4         964868     9506 2021-04-30   6725.0        100.0
5476246   648596    12225 2021-08-13     76.0          0.0
5476247   546862     9673 2021-04-13   2308.0         49.0
5476248   697262    15297 2021-08-20  18307.0         63.0
5476249   384202    16197 2021-04-19   6203.0        100.0
5476250   319709     4436 2021-08-15   3921.0         45.0

In [ ]:
print(f"Interactions dataframe shape: {interactions_df.shape}")
print(f"Unique users in interactions: {interactions_df['user_id'].nunique():_}")
print(f"Unique items in interactions: {interactions_df['item_id'].nunique():_}")

Interactions dataframe shape: (5476251, 5)
Unique users in interactions: 962_179
Unique items in interactions: 15_706


In [ ]:
max_date = interactions_df['datetime'].max()
min_date = interactions_df['datetime'].min()

print(f"min date in interactions: {min_date}")
print(f"max date in interactions: {max_date}")

min date in interactions: 2021-03-13 00:00:00
max date in interactions: 2021-08-22 00:00:00


### users

In [ ]:
pd.concat([users.head(), users.tail()])

user_id         age        income  sex  kids_flg
0        973171   age_25_34  income_60_90    М         1
1        962099   age_18_24  income_20_40    М         0
2       1047345   age_45_54  income_40_60    Ж         0
3        721985   age_45_54  income_20_40    Ж         0
4        704055   age_35_44  income_60_90    Ж         0
840192   339025  age_65_inf   income_0_20    Ж         0
840193   983617   age_18_24  income_20_40    Ж         1
840194   251008         NaN           NaN  NaN         0
840195   590706         NaN           NaN    Ж         0
840196   166555  age_65_inf  income_20_40    Ж         0

In [ ]:
print(f"Users dataframe shape {users.shape}")
print(f"Unique users: {users['user_id'].nunique():_}")

Users dataframe shape (840197, 5)
Unique users: 840_197


### items

In [ ]:
pd.concat([items.head(2), items.tail(2)])

item_id content_type           title      title_orig  release_year  \
0        10711         film  Поговори с ней  Hable con ella        2002.0   
1         2508         film     Голые перцы    Search Party        2014.0   
15961     4538       series    Среди камней      Darklands         2019.0   
15962     3206       series            Гоша             NaN        2019.0   

                                        genres countries  for_kids  \
0      драмы, зарубежные, детективы, мелодрамы   Испания       NaN   
1             зарубежные, приключения, комедии       США       NaN   
15961                   драмы, спорт, криминал    Россия       0.0   
15962                                  комедии    Россия       0.0   

       age_rating studios                      directors  \
0            16.0     NaN               Педро Альмодовар   
1            16.0     NaN                 Скот Армстронг   
15961        18.0     NaN  Марк О’Коннор, Конор МакМахон   
15962        16.0     NaN                 Михаил Миронов   

                                                  actors  \
0      Адольфо Фернандес, Ана Фернандес, Дарио Гранди...   
1      Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...   
15961  Дэйн Уайт О’Хара, Томас Кэйн-Бирн, Джудит Родд...   
15962                 Мкртыч Арзуманян, Виктория Рунцова   

                                             description  \
0      Мелодрама легендарного Педро Альмодовара «Пого...   
1      Уморительная современная комедия на популярную...   
15961  Семнадцатилетний Дэмиен мечтает вырваться за п...   
15962  Добродушный Гоша не может выйти из дома, чтобы...   

                                                keywords  
0      Поговори, ней, 2002, Испания, друзья, любовь, ...  
1      Голые, перцы, 2014, США, друзья, свадьбы, прео...  
15961                        Среди, камней, 2019, Россия  
15962                                 Гоша, 2019, Россия

In [ ]:
print(f"Items dataframe shape {items.shape}")
print(f"Unique item_id: {items['item_id'].nunique():_}")

Items dataframe shape (15963, 14)
Unique item_id: 15_963


## UserKNN Model

In [ ]:
class UserKnn():
    """Class for fit-perdict UserKNN model
       based on ItemKNN model from implicit.nearest_neighbours
    """

    def __init__(self, model: ItemItemRecommender, N_users: int = 50):
        self.N_users = N_users
        self.model = model
        self.is_fitted = False

    def get_mappings(self, train):
        self.users_inv_mapping = dict(enumerate(train['user_id'].unique()))
        self.users_mapping = {v: k for k, v in self.users_inv_mapping.items()}

        self.items_inv_mapping = dict(enumerate(train['item_id'].unique()))
        self.items_mapping = {v: k for k, v in self.items_inv_mapping.items()}

    def get_matrix(self, df: pd.DataFrame,
                   user_col: str = 'user_id',
                   item_col: str = 'item_id',
                   weight_col: str = None,
                   users_mapping: Dict[int, int] = None,
                   items_mapping: Dict[int, int] = None):

        if weight_col:
            weights = df[weight_col].astype(np.float32)
        else:
            weights = np.ones(len(df), dtype=np.float32)

        self.interaction_matrix = sp.sparse.coo_matrix((
            weights,
            (
                df[item_col].map(self.items_mapping.get),
                df[user_col].map(self.users_mapping.get)
            )
            ))

        self.watched = df\
            .groupby(user_col, as_index=False)\
            .agg({item_col: list})\
            .rename(columns={user_col: 'sim_user_id'})

        return self.interaction_matrix

    def idf(self, n: int, x: float):
        return np.log((1 + n) / (1 + x) + 1)

    def _count_item_idf(self, df: pd.DataFrame):
        item_cnt = Counter(df['item_id'].values)
        item_idf = pd.DataFrame.from_dict(item_cnt, orient='index',
                                          columns=['doc_freq']).reset_index()
        item_idf['idf'] = item_idf['doc_freq'].apply(lambda x: self.idf(self.n, x))
        self.item_idf = item_idf

    def fit(self, train: pd.DataFrame):
        self.user_knn = self.model
        self.get_mappings(train)
        self.weights_matrix = self.get_matrix(train,
                                              users_mapping=self.users_mapping,
                                              items_mapping=self.items_mapping)

        self.n = train.shape[0]
        self._count_item_idf(train)

        self.user_knn.fit(self.weights_matrix)
        self.is_fitted = True

    def _generate_recs_mapper(self, model: ItemItemRecommender, user_mapping: Dict[int, int],
                              user_inv_mapping: Dict[int, int], N: int):
        def _recs_mapper(user):
            user_id = self.users_mapping[user]
            users, sim = model.similar_items(user_id, N=N)
            return [self.users_inv_mapping[user] for user in users], sim
        return _recs_mapper

    def predict(self, test: pd.DataFrame, N_recs: int = 10):

        if not self.is_fitted:
            raise ValueError("Please call fit before predict")

        mapper = self._generate_recs_mapper(
            model=self.user_knn,
            user_mapping=self.users_mapping,
            user_inv_mapping=self.users_inv_mapping,
            N=self.N_users
        )

        recs = pd.DataFrame({'user_id': test['user_id'].unique()})
        recs['sim_user_id'], recs['sim'] = zip(*recs['user_id'].map(mapper))
        recs = recs.set_index('user_id').apply(pd.Series.explode).reset_index()

        recs = recs[~(recs['user_id'] == recs['sim_user_id'])]\
            .merge(self.watched, on=['sim_user_id'], how='left')\
            .explode('item_id')\
            .sort_values(['user_id', 'sim'], ascending=False)\
            .drop_duplicates(['user_id', 'item_id'], keep='first')\
            .merge(self.item_idf, left_on='item_id', right_on='index', how='left')

        recs['score'] = recs['sim'] * recs['idf']
        recs = recs.sort_values(['user_id', 'score'], ascending=False)
        recs['rank'] = recs.groupby('user_id').cumcount() + 1
        return recs[recs['rank'] <= N_recs][['user_id', 'item_id', 'score', 'rank']]

    def predict_single(self, user_id: int, N_recs: int = 10) -> List[int]:
        """
        Predicts a list of item_id values for a given user using the fitted UserKNN model.

        Parameters:
        - user_id (int): The ID of the user for whom recommendations are requested.
        - N_recs (int, optional): Number of recommendations to return. Defaults to 10.

        Returns:
        - List[int]: A list of item_id values representing the recommended items.
        """
        # Check if the model is fitted
        if not self.is_fitted:
            raise ValueError("Please call fit before predict")

        # Map the user_id to the internal representation
        user_id_mapped = self.users_mapping.get(user_id)

        # Get similar items and their similarities for the given user
        users, sim = self.user_knn.similar_items(user_id_mapped, N=self.N_users)

        recs = []  # List to store recommended item_id values
        for sim_user, similarity in zip(users, sim):
            sim_user_id = self.users_inv_mapping.get(sim_user)
            # Exclude the user_id itself from recommendations
            if sim_user_id is not None and sim_user_id != user_id:
                # Get watched items for the similar user
                watched_items = self.watched.loc[self.watched['sim_user_id'] == sim_user_id, 'item_id'].values
                watched_items_flat = np.concatenate(watched_items)

                # Remove duplicates and None values
                watched_items_flat = np.unique(watched_items_flat[watched_items_flat != None])

                # Use map to get the mapped values of item_id
                recs.extend(filter(None, map(self.items_mapping.get, watched_items_flat)))

        # Limit to N_recs recommendations
        recs = list(set(recs))[:N_recs]
        return recs

## CV

In [ ]:
# Number of splits for time-based cross-validation
N_SPLITS = 3

# Time duration for each test set in the cross-validation
TEST_SIZE = '7D'  # 7 days

# Initializing a generator for time-based cross-validation folds
cv = TimeRangeSplitter(
    test_size=TEST_SIZE,
    n_splits=N_SPLITS,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)


In [ ]:
# Obtain the borders of test folds in the time-based cross-validation
test_fold_borders = cv.get_test_fold_borders(interactions)

In [ ]:
# Dictionary containing various recommendation metrics to evaluate the models
metrics = {
    'map@10': MAP(k=10),                # Mean Average Precision at 10
    'novelty': MeanInvUserFreq(k=10),   # Novelty based on the mean inverse user frequency at 10
    'serendipity': Serendipity(k=10),   # Serendipity at 10
    "precision@10": Precision(k=10),    # Precision at 10
    "recall@10": Recall(k=10),          # Recall at 10
}

# Dictionary containing several simple recommendation models for comparison
models = {
    'tfidf_userknn': TFIDFRecommender(),
    'bm25_userknn': BM25Recommender(),
}

# List of user count values for evaluation
N_users_values = [40, 60]

In [ ]:
%%time

results = []

# Generate time-based cross-validation folds
fold_iterator = cv.split(interactions, collect_fold_stats=True)

# Iterate over each fold
for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
    print(f"\n==================== Fold {i_fold}")
    pprint(fold_info)

    # Extract training and test sets for the current fold
    df_train = interactions.df.iloc[train_ids].copy()
    df_test = interactions.df.iloc[test_ids][Columns.UserItem].copy()

    # Create a catalog of unique items in the training set
    catalog = df_train[Columns.Item].unique()

    # Iterate over each user count and each model for evaluation
    for N_users in N_users_values:
        for model_name, model in models.items():
            # Create and fit a UserKnn model with the specified number of users (N_users)
            userknn_model = UserKnn(model=model, N_users=N_users)
            userknn_model.fit(df_train)

            # Generate recommendations for the test set
            recos = userknn_model.predict(df_test)

            # Calculate metric values for the current model, fold, and user count
            metric_values = calc_metrics(
                metrics,
                reco=recos,
                interactions=df_test,
                prev_interactions=df_train,
                catalog=catalog,
            )

            # Create a dictionary containing fold information, model name, user count, and metric values
            fold = {"fold": i_fold, "model": model_name, "N_users": N_users}
            fold.update(metric_values)
            results.append(fold)


==================== Fold 0
{'end': Timestamp('2021-08-09 00:00:00', freq='7D'),
 'i_split': 0,
 'start': Timestamp('2021-08-02 00:00:00', freq='7D'),
 'test': 263681,
 'test_items': 6602,
 'test_users': 98184,
 'train': 4266013,
 'train_items': 15237,
 'train_users': 797423}


/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.35912609100341797 seconds
  warnings.warn(


  0%|          | 0/797423 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.379563570022583 seconds
  warnings.warn(


  0%|          | 0/797423 [00:00<?, ?it/s]

...

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.44608044624328613 seconds
  warnings.warn(


  0%|          | 0/906071 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.5245273113250732 seconds
  warnings.warn(


  0%|          | 0/906071 [00:00<?, ?it/s]

CPU times: user 11h 4min 31s, sys: 3min 47s, total: 11h 8min 18s
Wall time: 7h 14min 46s


## Metrics

In [ ]:
# Create a pandas DataFrame from the results list
df_metrics = pd.DataFrame(results)

# Display the DataFrame containing evaluation metrics
df_metrics

fold          model  N_users  precision@10  recall@10    map@10   novelty  \
0      0  tfidf_userknn       40      0.006922   0.035573  0.006773  7.573736   
1      0   bm25_userknn       40      0.003119   0.014228  0.002784  9.199335   
2      0  tfidf_userknn       60      0.006922   0.035573  0.006773  7.573736   
3      0   bm25_userknn       60      0.003119   0.014228  0.002784  9.199335   
4      1  tfidf_userknn       40      0.006410   0.033086  0.006279  7.631293   
5      1   bm25_userknn       40      0.003011   0.013753  0.002679  9.285580   
6      1  tfidf_userknn       60      0.006410   0.033086  0.006279  7.631293   
7      1   bm25_userknn       60      0.003011   0.013753  0.002679  9.285580   
8      2  tfidf_userknn       40      0.006257   0.031099  0.005949  7.710766   
9      2   bm25_userknn       40      0.003020   0.013504  0.002633  9.364591   
10     2  tfidf_userknn       60      0.006257   0.031099  0.005949  7.710766   
11     2   bm25_userknn       60      0.003020   0.013504  0.002633  9.364591   

    serendipity  
0      0.000061  
1      0.000090  
2      0.000061  
3      0.000090  
4      0.000066  
5      0.000102  
6      0.000066  
7      0.000102  
8      0.000070  
9      0.000105  
10     0.000070  
11     0.000105

In [ ]:
# Group the DataFrame by the 'model' column and calculate the mean for each metric
model_metrics_mean = df_metrics.groupby(['model', 'N_users']).mean()[metrics.keys()]

# Display the mean metric values for each model
model_metrics_mean

map@10   novelty  serendipity  precision@10  \
model         N_users                                                  
bm25_userknn  40       0.002699  9.283169     0.000099      0.003050   
              60       0.002699  9.283169     0.000099      0.003050   
tfidf_userknn 40       0.006334  7.638598     0.000065      0.006529   
              60       0.006334  7.638598     0.000065      0.006529   

                       recall@10  
model         N_users             
bm25_userknn  40        0.013828  
              60        0.013828  
tfidf_userknn 40        0.033253  
              60        0.033253

## Model saving

In [ ]:
# Create a UserKnn model with TFIDFRecommender as the base model and 50 users
userknn = UserKnn(model=TFIDFRecommender(), N_users=50)

# Fit the UserKnn model to the entire interactions DataFrame
userknn.fit(interactions.df)

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.46730756759643555 seconds
  warnings.warn(


  0%|          | 0/962179 [00:00<?, ?it/s]

In [ ]:
# Save the UserKnn model to a file using dill
with open('userknn.dill', 'wb') as f:
    dill.dump(userknn, f)

In [ ]:
# Load the UserKnn model from the saved file
with open('userknn.dill', 'rb') as f:
    userknn = dill.load(f)

# Example of finding similar items to the user with internal id = 11
userknn.predict_single(11)

[673, 2337, 646, 456, 142, 1071, 16, 112, 174, 148]